In [1]:
# --- CELL 1: SETUP MÔI TRƯỜNG & SỬA LỖI ---
import os

# # 1. Gỡ và cài lại Pydantic bản thấp để tránh xung đột Gradio
# !pip install "pydantic<2.12" --force-reinstall

# 2. Cài Datasets bản 2.21.0 (Bản ổn định nhất hiện tại trên Kaggle)
!pip install "datasets==2.21.0" --force-reinstall

# 3. Cài lại WandB mới nhất để tránh lỗi circular import
!pip install "wandb" --upgrade --force-reinstall

# 4. Cài các thư viện Training cần thiết
!pip install -q transformers evaluate accelerate librosa gradio kagglehub

print("✅ Đã cài đặt xong môi trường. Sẵn sàng chạy code chính!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 4.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7

In [2]:
# --- CELL 2: FULL TRAINING PIPELINE ---

import os # <--- Đã import os để tránh lỗi NameError
import glob
import numpy as np
import pandas as pd
import librosa
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from datasets import Dataset, Audio, ClassLabel, Features
import evaluate
from transformers import (
    AutoFeatureExtractor, 
    AutoModelForAudioClassification, 
    TrainingArguments, 
    Trainer
)
from kaggle_secrets import UserSecretsClient
import wandb

# ==========================================
# 1. KẾT NỐI WANDB (Tự động lấy Key)
# ==========================================
try:
    wandb.login(key="8b070d287fa4a783a07c13c8967eec041a6eaf59")
    print("✅ Đã đăng nhập WandB thành công!")
except Exception as e:
    print("⚠️ Không tìm thấy Secret WandB. Chạy chế độ Offline.")
    os.environ["WANDB_MODE"] = "disabled"

# ==========================================
# 2. LOAD DỮ LIỆU TỪ KAGGLE INPUT
# ==========================================
print("\n📂 Đang quét dữ liệu trong /kaggle/input ...")
wav_files = glob.glob('/kaggle/input/**/*.wav', recursive=True)

if not wav_files:
    raise ValueError("❌ Không tìm thấy file wav nào! Hãy Add Data CREMA-D vào notebook.")
else:
    print(f"✅ Tìm thấy {len(wav_files)} file âm thanh.")

data = []
# Mapping nhãn CREMA-D
label_map = {
    'ANG': 'angry', 'DIS': 'disgust', 'FEA': 'fear', 
    'HAP': 'happy', 'NEU': 'neutral', 'SAD': 'sad'
}

for file_path in wav_files:
    filename = os.path.basename(file_path)
    parts = filename.split('_')
    
    if len(parts) >= 3:
        emotion_code = parts[2]
        if emotion_code in label_map:
            data.append({
                'file': file_path, 
                'label': label_map[emotion_code]
            })

df = pd.DataFrame(data)
print(f"📊 DataFrame shape: {df.shape}")

# Chia Train/Test (80/20)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Tạo danh sách nhãn
class_names = sorted(list(set(df['label'])))
label2id = {label: str(i) for i, label in enumerate(class_names)}
id2label = {str(i): label for i, label in enumerate(class_names)}

# Chuyển sang HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# ==========================================
# 3. PREPROCESSING (Thủ công với Librosa)
# ==========================================
model_id = "openai/whisper-base"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)

def preprocess_function(examples):
    file_paths = examples['file']
    audio_arrays = []
    
    for path in file_paths:
        # Load file bằng librosa để tránh lỗi backend của datasets
        # sr=16000 là bắt buộc cho Whisper
        array, _ = librosa.load(path, sr=16000)
        audio_arrays.append(array)
    
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=16000, 
        max_length=int(16000 * 30), # Max 30s
        truncation=True,
        return_tensors="pt"
    )
    
    if 'label' in examples:
        labels = [int(label2id[l]) for l in examples['label']]
        inputs["labels"] = labels
        
    return inputs

print("\n🚀 Đang xử lý dữ liệu (Feature Extraction)...")
encoded_train = train_dataset.map(preprocess_function, batched=True, batch_size=32, remove_columns=train_dataset.column_names)
encoded_test = test_dataset.map(preprocess_function, batched=True, batch_size=32, remove_columns=test_dataset.column_names)

# ==========================================
# 4. CẤU HÌNH TRAINING
# ==========================================
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

model = AutoModelForAudioClassification.from_pretrained(
    model_id, 
    num_labels=len(class_names),
    label2id=label2id,
    id2label=id2label
)

training_args = TrainingArguments(
    output_dir="/kaggle/working/whisper_crema_output",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    
    # --- Cấu hình tối ưu ---
    per_device_train_batch_size=32,  # Batch size lớn cho P100/T4
    per_device_eval_batch_size=32,
    dataloader_num_workers=4,        # Tận dụng CPU Kaggle
    gradient_accumulation_steps=1,
    
    # --- Epoch & Regularization ---
    num_train_epochs=8,              # Train cứng 8 epochs (Không dùng EarlyStopping)
    weight_decay=0.01,               # Giảm Overfitting
    load_best_model_at_end=True,     # Lưu lại model tốt nhất (dựa trên Accuracy hoặc Loss)
    metric_for_best_model="loss",    # Chọn model có Loss thấp nhất
    greater_is_better=False,
    
    warmup_ratio=0.1,
    logging_steps=10,
    fp16=True,                       # Mixed Precision
    save_total_limit=1,              # Chỉ giữ 1 checkpoint nhẹ máy
    report_to="wandb",
    run_name="crema-whisper-base-final"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

# ==========================================
# 5. CHẠY TRAINING
# ==========================================
print("\n🔥 Bắt đầu Training...")
trainer.train()

# Kết thúc log WandB
wandb.finish()

# ==========================================
# 6. ĐÁNH GIÁ & VẼ CONFUSION MATRIX
# ==========================================
print("\n📊 Đang đánh giá chi tiết trên tập Test...")

# Dự đoán
predictions_output = trainer.predict(encoded_test)
y_preds = np.argmax(predictions_output.predictions, axis=1)
y_true = predictions_output.label_ids

# Vẽ Matrix
cm = confusion_matrix(y_true, y_preds)
plt.figure(figsize=(10, 8))
labels = [id2label[str(i)] for i in range(len(class_names))]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Dự đoán (Predicted)')
plt.ylabel('Thực tế (Actual)')
plt.title('Confusion Matrix')
plt.show()

# In báo cáo text
print("\n📑 Classification Report:")
print(classification_report(y_true, y_preds, target_names=labels))

2025-12-24 09:32:34.275008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766568754.466694      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766568754.522441      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766568754.989757      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766568754.989792      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766568754.989795      24 computation_placer.cc:177] computation placer alr

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

wandb: setting up run b045aczf
wandb: Tracking run with wandb version 0.23.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251224_093724-b045aczf
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run crema-whisper-base-final
wandb: ⭐️ View project at https://wandb.ai/duchung04st-hanoi-university-of-science-and-technology/huggingface
wandb: 🚀 View run at https://wandb.ai/duchung04st-hanoi-university-of-science-and-technology/huggingface/runs/b045aczf


Epoch,Training Loss,Validation Loss,Accuracy
1,1.014500,1.045396,0.638684
2,0.729300,0.806979,0.702485
3,0.603500,0.823205,0.719946
4,0.368200,0.766963,0.741437
5,0.184600,0.885697,0.740766
6,0.137100,1.017407,0.728677
7,0.052100,1.138391,0.742780
8,0.013000,1.200354,0.740766


/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:461: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50


📊 Đang đánh giá chi tiết trên tập Test...


Error: You must call wandb.init() before wandb.log()

In [ ]:
!pip install -U datasets

In [ ]:
# --- CELL 2: OPTIMIZED TRAINING PIPELINE ---

import os
import glob
import numpy as np
import pandas as pd
import librosa
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GroupShuffleSplit # <-- Dùng để chia theo Actor
from sklearn.metrics import confusion_matrix, classification_report
from datasets import Dataset
import evaluate
from transformers import (
    AutoFeatureExtractor, 
    AutoModelForAudioClassification, 
    TrainingArguments, 
    Trainer,
    EarlyStoppingCallback
)
import wandb
from kaggle_secrets import UserSecretsClient

# ==========================================
# 1. KẾT NỐI WANDB (BẢO MẬT HƠN)
# ==========================================
# Khuyên dùng: Add Key vào Add-ons -> Secrets trên Kaggle với label là "wandb_key"
try:
    user_secrets = UserSecretsClient()
    wandb_key = user_secrets.get_secret("wandb_key")
    wandb.login(key="8b070d287fa4a783a07c13c8967eec041a6eaf59")
    print("✅ Đã đăng nhập WandB thành công!")
except:
    print("⚠️ Không tìm thấy Secret. Đang dùng hardcoded key (Không khuyến khích)...")
    # Fallback nếu bạn lười add secret (nhưng hãy cẩn thận lộ key)
    os.environ["WANDB_API_KEY"] = "8b070d287fa4a783a07c13c8967eec041a6eaf59"
    wandb.login()

# ==========================================
# 2. LOAD DỮ LIỆU & CHIA THEO ACTOR
# ==========================================
print("\n📂 Đang quét dữ liệu...")
wav_files = glob.glob('/kaggle/input/**/*.wav', recursive=True)

data = []
label_map = {
    'ANG': 'angry', 'DIS': 'disgust', 'FEA': 'fear', 
    'HAP': 'happy', 'NEU': 'neutral', 'SAD': 'sad'
}

for file_path in wav_files:
    filename = os.path.basename(file_path)
    parts = filename.split('_')
    
    if len(parts) >= 3:
        actor_id = parts[0] # CREMA-D filename: 1001_DFA_ANG_XX.wav -> Actor 1001
        emotion_code = parts[2]
        if emotion_code in label_map:
            data.append({
                'file': file_path, 
                'label': label_map[emotion_code],
                'actor_id': actor_id # <-- Lưu thêm Actor ID
            })

df = pd.DataFrame(data)

# --- CHIA TRAIN/TEST THEO ACTOR (QUAN TRỌNG) ---
# Đảm bảo giọng người trong tập Test chưa từng xuất hiện trong Train
splitter = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_idxs, test_idxs = next(splitter.split(df, groups=df['actor_id']))

train_df = df.iloc[train_idxs].reset_index(drop=True)
test_df = df.iloc[test_idxs].reset_index(drop=True)

print(f"📊 Train set: {len(train_df)} samples")
print(f"📊 Test set: {len(test_df)} samples (Actors hoàn toàn mới)")

# Tạo mapping nhãn
class_names = sorted(list(set(df['label'])))
label2id = {label: str(i) for i, label in enumerate(class_names)}
id2label = {str(i): label for i, label in enumerate(class_names)}

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# ==========================================
# 3. PREPROCESSING & AUGMENTATION
# ==========================================
model_id = "openai/whisper-base"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)

def add_noise(audio, noise_factor=0.005):
    """Thêm nhiễu trắng nhẹ để tránh Overfitting"""
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    return augmented_audio

def preprocess_function(examples, is_training=False):
    file_paths = examples['file']
    audio_arrays = []
    
    for path in file_paths:
        # Load audio
        array, _ = librosa.load(path, sr=16000)
        
        # Chỉ Augment (thêm nhiễu) nếu là tập Train
        if is_training:
            array = add_noise(array)
            
        audio_arrays.append(array)
    
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=16000, 
        max_length=16000 * 30, # Whisper yêu cầu 30s
        padding="max_length",  # Pad đều
        truncation=True,
        return_tensors="pt"
    )
    
    if 'label' in examples:
        inputs["labels"] = [int(label2id[l]) for l in examples['label']]
        
    return inputs

print("\n🚀 Đang xử lý dữ liệu...")
# Map Train có augmentation
encoded_train = train_dataset.map(lambda x: preprocess_function(x, is_training=True), batched=True, batch_size=16, remove_columns=train_dataset.column_names)
# Map Test giữ nguyên gốc
encoded_test = test_dataset.map(lambda x: preprocess_function(x, is_training=False), batched=True, batch_size=16, remove_columns=test_dataset.column_names)

# ==========================================
# 4. CẤU HÌNH TRAINING (CHỐNG OVERFITTING)
# ==========================================
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

model = AutoModelForAudioClassification.from_pretrained(
    model_id, 
    num_labels=len(class_names),
    label2id=label2id,
    id2label=id2label
)

training_args = TrainingArguments(
    output_dir="/kaggle/working/whisper_crema_final",
    
    # --- Evaluation ---
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="loss", # Dừng dựa trên Loss
    
    # --- Hyperparameters ---
    learning_rate=2e-5,          # Giảm nhẹ LR (gốc 3e-5) để hội tụ mượt hơn
    per_device_train_batch_size=16, # Giảm batch size nếu OOM (Whisper Base khá nặng)
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2, # Bù lại batch size nhỏ
    
    # --- Regularization ---
    num_train_epochs=1,         # Tăng epoch nhưng dùng EarlyStopping
    weight_decay=0.1,            # Tăng mạnh Weight Decay (gốc 0.01) -> Chống Overfit
    warmup_ratio=0.1,
    
    logging_steps=20,
    fp16=True,
    save_total_limit=2,
    report_to="wandb",
    run_name="crema-whisper-optimized"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    # --- Thêm Callback Dừng sớm ---
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] 
    # Nếu Loss không giảm trong 3 epochs liên tiếp -> Dừng ngay
)

# ==========================================
# 5. CHẠY TRAINING
# ==========================================
print("\n🔥 Bắt đầu Training...")
trainer.train()
wandb.finish()

# ==========================================
# 6. ĐÁNH GIÁ KẾT QUẢ
# ==========================================
print("\n📊 Đang đánh giá...")
predictions = trainer.predict(encoded_test)
y_preds = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# Confusion Matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true, y_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[id2label[str(i)] for i in range(len(class_names))],
            yticklabels=[id2label[str(i)] for i in range(len(class_names))])
plt.title('Confusion Matrix (Unseen Actors)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

print(classification_report(y_true, y_preds, target_names=[id2label[str(i)] for i in range(len(class_names))]))